In [1]:
import time
import adodbapi
import pandas as pd
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
prodORitem="SKU"

client_manuf =["Malard Nicolas"]
client_brands = ["Nicolas"]

categories =["Effervescents"]
sectors = ["Champagnes","Mousseux","Effervescents Sans Alcool"]
segments = ["Blanc De Blancs","Bruts","Crus","Extra Brut","Millesimes","Rosés","Mousseux Bourgogne","Mousseux Italiens","Mousseux Val De Loire","Mousseux Alsace","Mousseux Bordelais"]
subsegments= []
subcategories= []

decimals = 2
sign = "after"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

national = False
customareas= "REVENUE"
areas = ["CHANNEL","REGION", f'{customareas}']


regions_RET  = []
channels_RET = []
market_RET = []
 
regions_CHAN = ["NICOLAS"]
channels_CHAN = ["NICOLAS QCN","NICOLAS VCN","NICOLAS QCT","NICOLAS QCA","NICOLAS CCP"]#"NICOLAS VCT","NICOLAS RN","NICOLAS CCQ","NICOLAS CCC"]#
market_CHAN = []

regions_REG = []
channels_REG = ["NICOLAS IDF","NICOLAS PAC","NICOLAS RHO","NICOLAS AQU","NICOLAS EST"]#"NICOLAS OCC","NICOLAS HAU","NICOLAS NDI","NICOLAS BRE","NICOLAS LOI","NICOLAS CEN","NICOLAS BOU"]#
market_REG = []

regions_CUST = []
channels_CUST = ["CA A","CA B","CA C","CA D","CA E"]
market_CUST = []

 
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending June 2025"
end_date = "2025-07-01"
years = ['2023', '2024','2025']

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date, periods=3, freq='ME').strftime('%b-%y').tolist()
past_36_months = pd.date_range(end=end_date, periods=36, freq='ME').strftime('%b-%y').tolist()


 
National=["NATIONAL"] if national else []

entity_hierarchy = [
    ("Region", "RETAILER", regions_RET),
    ("Channel", "RETAILER", channels_RET),
    ("Market", "RETAILER",  market_RET),
    ("Region", "CHANNEL",regions_CHAN),
    ("Channel", "CHANNEL",channels_CHAN),
    ("Market", "CHANNEL",market_CHAN),
   ("Region", "REGION",regions_REG),
   ("Channel","REGION",channels_REG),
   ("Market","REGION", market_REG),
   ("Region",customareas, regions_CUST),
    ("Channel", customareas, channels_CUST),
    ("Market", customareas, market_CUST)
]

hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]


direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

server = "powerbi://api.powerbi.com/v1.0/myorg/Nicolas"
dataset_name = "Nicolas Sparkling Dataset"
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"
p3m_dax = "{" + ", ".join(f'"{date}"' for date in past_3_months) + "}"
p36m_dax = "{" + ", ".join(f'"{date}"' for date in past_36_months) + "}"
normalized = True
promo_type = True
dispaly_share = False  # True if Available
feature_share = False

print(p12m_dax)
path=os.path.join(os.getcwd(),"Promotion Datasets NewEX")

conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=900;"
print(conn_str)

{"Jul-24", "Aug-24", "Sep-24", "Oct-24", "Nov-24", "Dec-24", "Jan-25", "Feb-25", "Mar-25", "Apr-25", "May-25", "Jun-25"}
Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;


In [3]:
entity_hierarchy = [
    item for item in entity_hierarchy
    if len(item) == 3 and item[2]  # item[2] is the entity list
]
month_years =  pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST
promo_col = []
promo_col = promo_col+['Display Share'] if dispaly_share else promo_col
promo_col = promo_col+['Feature Share'] if feature_share else promo_col

C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_14828\656077122.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_years =  pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()


In [4]:
start_time = time.time()

## By Promo Types

#### Brands and Promo Types

In [5]:
def execute_dax_query(BrandorManuf, entity_name, market, area, hierby, entity_type):
    outputdic = {}
    columns = ["Value Share", "Promo Sales"]
    column_exprs = ",\n            ".join(f'"{col}", COALESCE(CALCULATE([{col}]), 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            CROSSJOIN(
                VALUES(Products[{BrandorManuf}]),
                VALUES('Promo Description'[Promo Type])
            ),
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{hierby}] = "{entity_type}",
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({p12m_dax}, 'Calendar'[MonthYear]),
        TREATAS({{"{entity_name}"}}, 'Market'[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}")
    )
    """
    parentdax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            VALUES(Products[{BrandorManuf}]),
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{hierby}] = "{entity_type}",
        TREATAS({p12m_dax}, 'Calendar'[MonthYear]),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{"{entity_name}"}}, 'Market'[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}")
    )
    """
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            VALUES(Products[Category]),
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{hierby}] = "{entity_type}",
        TREATAS({p12m_dax}, 'Calendar'[MonthYear]),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{"{entity_name}"}}, 'Market'[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}")
    )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            cols = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parentdax_query)
            pcols = [desc[0] for desc in cursor.description]
            pdata = cursor.fetchall()    

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            gcols = [desc[0] for desc in cursor.description]
            gdata = cursor.fetchall()

        # Format results into DataFrames
        df = pd.DataFrame(data, columns=cols)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        pdf = pd.DataFrame(pdata, columns=pcols)
        pdf.columns = pdf.columns.str.replace(r'.*\[|\]', '', regex=True)
        pdf = pdf.loc[~(pdf.select_dtypes(include='number') == 0).all(axis=1)]
        if not df.empty:
            if pdf.shape[1] > 1:
                pdf.iloc[:, 0] = pdf.iloc[:, 0].astype(str) + " Total"

            if not pdf.empty:
                df = pd.concat([df,pdf], ignore_index=True)
            dt = pd.DataFrame([gdata[0]], columns=[col.replace(']', '').split('[')[-1] for col in gcols])
            dt = dt[[col for col in dt.columns if col in df.columns]]

            missing_cols = [col for col in df.columns if col not in dt.columns]
            for col in missing_cols:
                dt[col] = pd.NA
            dt = dt[df.columns]
            dt[df.columns[0]] = 'Grand Total'
            
            df = pd.concat([df, dt], ignore_index=True)

            key = f"{entity_type} | {entity_name}"
            outputdic[key] = df
        print(f"Query executed successfully for {entity_type} {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")

    return outputdic

# List of all entities to process

brands_client_dfs = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for market,area,entity_list in entity_hierarchy:
        for entity in entity_list:
            for hierby, hier_values in hierarchy_levels:
                    for value in hier_values:
                        futures.append(executor.submit(execute_dax_query, f"{BrandOrTopB}",entity,market, area, hierby, value))
                
        # Wait for all tasks to complete
        for future in futures:
            result=future.result()
            brands_client_dfs.update(result)

pd.to_pickle(brands_client_dfs, os.path.join(path,"brands_promo_type.pkl"))


Database error for Effervescents Sans Alcool NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', None, None, 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for Effervescents NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', None, None, 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for Millesimes NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', None, None, 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for Rosés NICOLAS: (co

#### By Brands For P12M

In [6]:
def execute_dax_query(entity_name, market, area,hierby, entity_type):
    outputdic = {}
    if normalized == True:
        columns=["Promo Value", 'VSOD', 'VSOD IYA','Value Share','Promo Share','Value Uplift (v. base) Normalized','Value Uplift Normalized IYA','Volume Uplift (v. Base) Normalized', 'Volume Uplift Normalized IYA'] + promo_col

    else:        
        columns=["Promo Value", 'VSOD', 'VSOD IYA','Value Share','Promo Share','Value Uplift (v. base)','Value Uplift IYA','Volume Uplift (v. Base)', 'Volume Uplift IYA'] + promo_col


    column_exprs = ",\n            ".join(f'"{col}", COALESCE(CALCULATE([{col}]), 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandOrTopB}],  
            Products[Total Size],  
            
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",        
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS(
            {{"{entity_name}"}},
            Market[{market}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"     
        )
    )   
    """
    parentdax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandOrTopB}],              
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",        
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS(
            {{"{entity_name}"}},
            Market[{market}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{hierby}],
            
            {column_exprs}
        ),
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        
        TREATAS(
            {{"{entity_name}"}},
            Market[{market}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "Category"   
        )
    )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            cols = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parentdax_query)
            parentcols = [desc[0] for desc in cursor.description]
            parentdata = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            gcols = [desc[0] for desc in cursor.description]
            gdata = cursor.fetchall()

        # Format results into DataFrames
        df = pd.DataFrame(data, columns=cols)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        if not df.empty: 
            parentdf = pd.DataFrame(parentdata, columns=parentcols)
            parentdf.columns = parentdf.columns.str.replace(r'.*\[|\]', '', regex=True)
            parentdf = parentdf.loc[~(parentdf.select_dtypes(include='number') == 0).all(axis=1)]
            if parentdf.shape[1] > 1:
                parentdf.iloc[:, 0] = parentdf.iloc[:, 0].astype(str) + " Total"


                df = pd.concat([df,parentdf], ignore_index=True)
        
            dt = pd.DataFrame([gdata[0]], columns=[col.replace(']', '').split('[')[-1] for col in gcols])
            dt[df.columns[0]] = 'Grand Total'
            # Keep only columns that exist in df
            dt = dt.loc[:, dt.columns.isin(df.columns)]

            # Ensure column order matches df
            dt = dt.reindex(columns=df.columns)

            df = pd.concat([df, dt], ignore_index=True)

            key = f"{entity_type} | {entity_name}"
            outputdic[key] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")

    return outputdic

# List of all entities to process

promotions_brands_P12M = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for market, area,entity_list in entity_hierarchy:
        for entity in entity_list:
            for hierby, hier_values in hierarchy_levels:
                    for value in hier_values:
                        futures.append(executor.submit(execute_dax_query,entity, market, area,hierby, value))
                
    # Wait for all tasks to complete
        for future in futures:
            result=future.result()
            promotions_brands_P12M.update(result)

pd.to_pickle(promotions_brands_P12M, os.path.join(path,"promotions_brands_P12M.pkl"))


Database error for Effervescents NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', None, None, 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')Database error for Mousseux NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')

Database error for Crus NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data So

# VSOD

In [7]:
def execute_dax_query(entity_name, market, area, hierby,direct_parent):
    outputdic = {}
    key = f"{categories[0]} | {entity_name}"
    columns = [
        "VSOD"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)


    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}],
                    Products[{hierby}]
                    
                ),
                {column_exprs}
            ),
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}]                   
                ),
                {column_exprs}
            ),
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SELECTCOLUMNS(
                FILTER('Scope', 'Scope'[Scope] = "Category"),
                "Scope", 'Scope'[Scope]
            ),
            "VSOD", COALESCE([VSOD], 0)
        ),
        TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
    )
    """



    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"


            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan

            # Reorder columns exactly as in df
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category" :
                    continue
            if isinstance(hier_values, list):
                for market, area,entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key = f"{categories[0]} | {entity}"
                        ordered_keys.append(key)
                        future = executor.submit(
                            execute_dax_query, entity, market, area,hierby,direct_parent
                        )
                        futures[future] = key
      

            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            filename = f"{hierby}_VSOD.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent)


Database error for NICOLAS VCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')Database error for NICOLAS QCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')

Database error for NICOLAS QCT in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HEL

# VSOD Client

In [8]:
def execute_dax_query(ManuforBrand,entity_name, market, area,hierby,direct_parent,client_list):
    outputdic = {}
    key = f"{categories[0]} | {entity_name}"

    columns = [
        "VSOD"
    ]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    client_values_dax = ", ".join([f'"{c.replace("\"", "\"\"")}"' for c in client_list])
    cat_filter = ""
    if hierby !="Sector":
        cat_filter = f'''
            FILTER(
                Products,
                Products[Category] = "{categories[0]}"
            ),
        '''    

    # Prepare client brand filter conditionally

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}],
                    Products[{hierby}],
                    Products[{ManuforBrand}]
                      
                ),
                {column_exprs}
            ),
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            {cat_filter}
            FILTER(
            Products,
            Products[{ManuforBrand}] IN {{ {client_values_dax} }}
              ),

            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}],
                    Products[{ManuforBrand}]
                                       
                ),
                {column_exprs}
            ),
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {cat_filter}
            FILTER(
            Products,
            Products[{ManuforBrand}] IN {{ {client_values_dax} }}
              ),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    childtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]                   
                ),
                {column_exprs}
            ),
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER(
            Products,
            Products[{ManuforBrand}] IN {{ {client_values_dax} }}
              ),
              {cat_filter}
            FILTER('Market', 'Market'[Area] = "{area}"), 
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SELECTCOLUMNS(
                FILTER('Scope', 'Scope'[Scope] = "Category"),
                "Scope", 'Scope'[Scope]
            ),
            "VSOD", COALESCE([VSOD], 0)
        ),
        
        TREATAS({p12m_dax}, Calendar[MonthYear]),
                    FILTER(
            Products,
            Products[{ManuforBrand}] IN {{ {client_values_dax} }}
              ),
            {cat_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
    )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(childtotal_dax_query)
            childtotal_columns = [desc[0] for desc in cursor.description]
            childtotal_data = cursor.fetchall()         
              
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        childtotal_df = pd.DataFrame(childtotal_data, columns=childtotal_columns)
        childtotal_df.columns = childtotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        childtotal_df = childtotal_df.loc[~(childtotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan
                    
        if childtotal_df.shape[1] > 1:
            childtotal_df.iloc[:, 0] =  childtotal_df.iloc[:, 0].astype(str) + " Total"


            # Reorder columns exactly as in df
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)
            df = pd.concat([df, childtotal_df], ignore_index=True)
            

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(ManuforBrand,entity_hierarchy, hierarchy_levels,direct_parent,client_list):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category" :
                    continue
            if isinstance(hier_values, list):
                for market, area,entity_list in entity_hierarchy:
                    for entity in entity_list:
                        for client in client_brands:
                            key = f"{categories[0]} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query,ManuforBrand, entity,market, area, hierby,direct_parent,client_list
                            )
                            futures[future] = key


            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if ManuforBrand==f'{BrandOrTopB}':
                filename = f"{hierby}_client_VSOD.pkl"
            else:
                filename = f"{hierby}_manuf_VSOD"
                 
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,direct_parent,client_brands)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,direct_parent,client_manuf)



Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS QCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS QCA in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\A

# By Products/Item P12M

In [9]:
def execute_dax_query(entity_name, market, area,hierby, entity_type):
    outputdic = {}
    if normalized == True:
        columns = ["Promo Sales","Promo Value","Discount Depth (%)","Promo Share","VSOD","Base Price/Unit","Promo Price/Unit","Gross Margin %","Value Uplift (v. base) Normalized",'Volume Uplift (v. Base) Normalized',"Trade Effectiveness","Incr Value"]
    else:        
        columns = ["Promo Sales","Promo Value","Discount Depth (%)","Promo Share","VSOD","Base Price/Unit","Promo Price/Unit","Gross Margin %","Value Uplift (v. base)",'Volume Uplift (v. Base)',"Trade Effectiveness","Incr Value"]

    column_exprs = ",\n            ".join(f'"{col}", COALESCE(CALCULATE([{col}]), 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandOrTopB}],  
            Products[{prodORitem}],  
            
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",        
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS(
            {{"{entity_name}"}},
            Market[{market}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            VALUES(Products[Category]),
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{hierby}] = "{entity_type}",
        TREATAS({p12m_dax}, 'Calendar'[MonthYear]),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{"{entity_name}"}}, 'Market'[{market}])
    )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            cols = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            gcols = [desc[0] for desc in cursor.description]
            gdata = cursor.fetchall()

        # Format results into DataFrames
        df = pd.DataFrame(data, columns=cols)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        dt = pd.DataFrame([gdata[0]], columns=[col.replace(']', '').split('[')[-1] for col in gcols])
        
        dt[df.columns[0]] = 'Grand Total'
        for col in df.columns:
            if col not in dt.columns:
                dt[col] = np.nan

        dt = dt[df.columns]
        df = pd.concat([df, dt], ignore_index=True)
        key = f"{entity_type} | {entity_name}"
        outputdic[key] = df
        print(f"Query executed successfully for {entity_type} {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")

    return outputdic

# List of all entities to process

promotions_products_P12M = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for market, area,entity_list in entity_hierarchy:
        for entity in entity_list:
            for hierby, hier_values in hierarchy_levels:
                    for value in hier_values:
                        futures.append(executor.submit(execute_dax_query,entity, market, area,hierby, value))
               
    # Wait for all tasks to complete
    for future in futures:
        result=future.result()
        promotions_products_P12M.update(result)

pd.to_pickle(promotions_products_P12M, os.path.join(path,"promotions_products_P12M.pkl"))


Database error for Effervescents NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')Database error for Effervescents Sans Alcool NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')

Database error for Mousseux NICOLAS: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\

# By End of The Week

In [10]:
def execute_dax_query(entity_name,entity_type,hierby, market, area,client,manuf):
    outputdic = {}
    key = f"{entity_type} | {client} | {entity_name}" if client else (f"{entity_type} | {manuf} | {entity_name}" if manuf else f"{entity_name}")
    if normalized == True:
        columns = ['Value Uplift (v. base) Normalized','Promo Value','Non Promo Value','Value Sales','Base Sales','VSOD','Discount Depth (%)','Promo Volume','Non Promo Volume']
    else:
        columns = ['Value Uplift (v. base)','Promo Value','Non Promo Value','Value Sales','Base Sales','VSOD','Discount Depth (%)','Promo Volume','Non Promo Volume']


    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    # Prepare client brand filter conditionally
    client_filter = ""
    if client:
        client_filter = f'''
            FILTER(
                Products,
                Products[Top Brands] = "{client}"
            ),
        '''    
    manuf_filter=''
    if  manuf:
        manuf_filter = f'''
            FILTER(
                Products,
                Products[Top Companies] = "{manuf}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Calendar,
                    Calendar[End of Week]
                    
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
 
        # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
            
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        # Rename the column if necessary (optional)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)

        # Find the column name for the date column (usually "End of Week")
        date_col = df.columns[0]  # Assuming the first column is the date
        if not df.empty:
            df[date_col] = pd.to_datetime(df[date_col]).dt.strftime('%Y-%m-%d')
        if not df.empty:
            df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0]).dt.date
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]



        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]

        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            # grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan

            # Reorder columns exactly as in df
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df, grand_tot], ignore_index=True)
        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for market, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            if client_manuf:
                                for manuf in client_manuf:
                                    key = f"{value} | {manuf} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(
                                        execute_dax_query, entity, value,hierby, market,area, '', manuf
                                    )
                                    futures[future] = key

                            if client_brands:
                                for client in client_brands:
                                    key = f"{value} | {client} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(
                                        execute_dax_query, entity, value,hierby, market,area, client, ''
                                    )
                                    futures[future] = key

        temp_results = {}
        for future in as_completed(futures):
            try:
                result = future.result()
                temp_results.update(result)
            except Exception as e:
                print(f"Error in future: {e}")

        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]

        filename = f"promotions_EndOfWeek"
        output_file = f"{path}\\{filename}.pkl"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

        print(f"All DataFrames for {hierby} saved to {output_file}.")


if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf,client_brands=client_brands)


Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS VCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO27

# Value Uplift vs Discount depth By  Client Brand

In [11]:
def execute_dax_query(entity_name,entity_type,hierby, market, area,client,manuf):
    outputdic = {}
    key = f"{entity_type} | {client} | {entity_name}" if client else (f"{entity_type} | {manuf} | {entity_name}" if manuf else f"{entity_name}")
    if normalized == True:
        columns = ['Discount Depth (%)','Promo Price/Unit','Value Uplift (v. base) Normalized','Promo Sales']
    else:
        columns = ['Discount Depth (%)','Promo Price/Unit','Value Uplift (v. base)','Promo Sales']


    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    client_filter = ""
    if client:
        client_filter = f'''
            FILTER(
                Products,
                Products[Top Brands] = "{client}"
            ),
        '''    
    manuf_filter=''
    if manuf:
        manuf_filter = f'''
            FILTER(
                Products,
                Products[Top Companies] = "{manuf}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
            CROSSJOIN(
                VALUES(Products[{prodORitem}]),
                VALUES('Calendar'[End of Week])
    
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
    parenttotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{prodORitem}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """
        # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_query)
            parcol = [desc[0] for desc in cursor.description]
            pardata = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        dp = pd.DataFrame(pardata, columns=parcol)
        dp.columns = dp.columns.str.replace(r'.*\[|\]', '', regex=True)
        dp = dp.loc[~(dp.select_dtypes(include='number') == 0).all(axis=1)]

        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]

        if dp.shape[1] > 1:
            dp.iloc[:, 0] = dp.iloc[:, 0].astype(str) + " Total"

        if dp.empty:
            outputdic[key] = dp
            return outputdic
        if not df.empty: 
            df = pd.concat([df, dp], ignore_index=True)

        if not grand_tot.empty and grand_tot.dropna(axis=1, how='all').shape[1] > 0:
            grand_tot[df.columns[0]] = 'Grand Total'
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan
            grand_tot = grand_tot[df.columns]

            # Final safety check to avoid warning
            if not grand_tot.dropna(how="all").empty:
                df = pd.concat([df, grand_tot], ignore_index=True)

            outputdic[key] = df
            print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for market, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            if client_manuf:
                                for manuf in client_manuf:
                                    key = f"{value} | {manuf} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(
                                        execute_dax_query, entity, value,hierby, market, area,'', manuf
                                    )
                                    futures[future] = key

                            if client_brands:
                                for client in client_brands:
                                    key = f"{value} | {client} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(
                                        execute_dax_query, entity, value,hierby, market, area,client, ''
                                    )
                                    futures[future] = key
    


        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if client_brands:
            filename = f"value_uplift"
        else:
            filename = f"value_uplift_manuf"

        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames for saved to {output_file}.")

if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands)
if client_manuf:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf)   


Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')Database error for NICOLAS QCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS QCA in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\AD

# Seasonality index dic

In [12]:
def execute_dax_query(entity_name,entity_type, market,area, hierby):
    outputdic = {}
    key = f"{entity_type} | {entity_name}"

    columns = ["Value Sales"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
            CROSSJOIN(
                VALUES(Products[{hierby}]),
                VALUES('Calendar'[MonthYear])
    
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            Products[{direct_parent[hierby]}] = "{entity_type}",

            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]                   
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
            
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:

        hierarchy_dict = dict(hierarchy_levels)
        for hierby, _ in hierarchy_levels:
            if hierby == "Category":
                continue
            dfs_results = {} 
            futures = {}
            ordered_keys = []
            # Get the parent level name (e.g., 'Segment')
            parent_level = direct_parent[hierby]

            # Get the list of values associated with that parent level
            parent_values = hierarchy_dict.get(parent_level, [])
            if isinstance(parent_values, list):
                for value in parent_values:
                    for market, area,entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{value} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query, entity, value, market, area,hierby
                            )
                            futures[future] = key
            
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            filename = f"{hierby}_MonthYear.pkl"

                 
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

process_dax_queries(entity_hierarchy, hierarchy_levels)



Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS VCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS RHO in REGION: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\AD

# Seasonality index dic Category

In [13]:
def execute_dax_query(entity_name, market,area,manuf):
    outputdic = {}
    key =f"{manuf} | {entity_name}"
    columns = ['Value Sales']

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    manuf_filter=''
    if manuf:
        manuf_filter = f'''
            FILTER(
                Products,
                Products[Top Companies] = "{manuf}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
            CROSSJOIN(
                VALUES(Products[Category]),
                VALUES('Calendar'[MonthYear])
    
                ),
                {column_exprs}
            ),
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SELECTCOLUMNS(
                FILTER('Scope', 'Scope'[Scope] = "Category"),
                "Scope", 'Scope'[Scope]
            ),
                {column_exprs}
            ),
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
        
        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
            
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

        outputdic[key] = df


        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy,client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        
        if client_manuf:
            for market, area,entity_list in entity_hierarchy:
                for entity in entity_list:
                    for manuf in client_manuf:
                        key = f"{manuf} | {entity}" 
                        ordered_keys.append(key)
                        future = executor.submit(
                            execute_dax_query, entity, market,area,manuf,
                        )
                        futures[future] = key               


            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            filename = f"Category_MonthYear"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            


if client_manuf:
    process_dax_queries(entity_hierarchy, client_manuf=client_manuf)   


Database error for NICOLAS VCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')Database error for NICOLAS in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\ADO270.CHM', 1240655, -2146824582), None), 'Error opening connection to "Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;"')
Database error for NICOLAS QCN in CHANNEL: (com_error(-2147352567, 'Exception occurred.', (0, 'ADODB.Connection', 'Provider cannot be found. It may not be properly installed.', 'C:\\WINDOWS\\HELP\\AD

In [14]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")


Script started at: Wed Sep 17 22:21:11 2025
Script ended at: Wed Sep 17 22:21:21 2025
Elapsed time: 10.00 seconds
